# Student Loan Risk with Deep Learning

In [76]:
# Imports
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from pathlib import Path

---

## Prepare the data to be used on a neural network model

### Step 1: Read the `student-loans.csv` file into a Pandas DataFrame. Review the DataFrame, looking for columns that could eventually define your features and target variables.   

In [77]:
# Read the csv into a Pandas DataFrame
file_path = "https://static.bc-edx.com/ai/ail-v-1-0/m18/lms/datasets/student-loans.csv"
loans_df = pd.read_csv(file_path)

# Review the DataFrame
loans_df.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score,credit_ranking
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0


In [78]:
# Review the data types associated with the columns
loans_df.dtypes

payment_history           float64
location_parameter        float64
stem_degree_score         float64
gpa_ranking               float64
alumni_success            float64
study_major_code          float64
time_to_completion        float64
finance_workshop_score    float64
cohort_ranking            float64
total_loan_score          float64
financial_aid_score       float64
credit_ranking              int64
dtype: object

In [79]:
# Check the credit_ranking value counts
loans_df["credit_ranking"].value_counts()

credit_ranking
1    855
0    744
Name: count, dtype: int64

### Step 2: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “credit_ranking”. The remaining columns should define the features dataset.

In [80]:
# Define the target set y using the credit_ranking column
y = loans_df["credit_ranking"]

# Display a sample of y
y.sample

<bound method NDFrame.sample of 0       0
1       0
2       0
3       1
4       0
       ..
1594    0
1595    1
1596    1
1597    0
1598    1
Name: credit_ranking, Length: 1599, dtype: int64>

In [81]:
# Define features set X by selecting all columns but credit_ranking
X = loans_df.drop('credit_ranking', axis=1)

# Review the features DataFrame
X

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4
...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2


### Step 3: Split the features and target sets into training and testing datasets.


In [82]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)


### Step 4: Use scikit-learn's `StandardScaler` to scale the features data.

In [83]:
# Create a StandardScaler instance
X_scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_scaler.fit(X_train)

# Fit the scaler to the features training dataset - DFQNOTE: this is repetitive. I think it should be "scale the data," which is what I did here
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

---

## Compile and Evaluate a Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [84]:
# Define the the number of inputs (features) to the model
input = len(X.columns)

# Review the number of features
input

11

In [85]:
# Define the number of hidden nodes for the first hidden layer
first_layer = tf.keras.layers.Dense(units = 5, activation = "relu", input_dim = input)

# Define the number of hidden nodes for the second hidden layer
second_layer = tf.keras.layers.Dense(units = 5, activation = "relu")

# Define the number of neurons in the output layer
output_layer = tf.keras.layers.Dense(units = 1, activation = "sigmoid")


c:\Users\Daniel Quinn\anaconda3\envs\dev\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [86]:
# Create the Sequential model instance
loan_model = tf.keras.models.Sequential()

# Add the first hidden layer
loan_model.add(first_layer)

# Add the second hidden layer
loan_model.add(second_layer)

# Add the output layer to the model specifying the number of output neurons and activation function
loan_model.add(output_layer)

In [87]:
# Display the Sequential model summary
loan_model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                 │ (None, 5)              │            60 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 5)              │            30 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 1)              │             6 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 96 (384.00 B)

 Trainable params: 96 (384.00 B)

 Non-trainable params: 0 (0.00 B)

### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [88]:
# Compile the Sequential model
loan_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [89]:
# Fit the model using 50 epochs and the training data
fit_loan_model = loan_model.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5420 - loss: 0.7167
Epoch 2/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5377 - loss: 0.7021
Epoch 3/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5386 - loss: 0.6834 
Epoch 4/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 844us/step - accuracy: 0.5913 - loss: 0.6741
Epoch 5/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6098 - loss: 0.6690 
Epoch 6/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6393 - loss: 0.6646 
Epoch 7/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6546 - loss: 0.6495
Epoch 8/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6944 - loss: 0.6341 
Epoch 9/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 781us/step - accuracy: 0.6996 - loss: 0.6314
Epoch 10/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6907 - loss: 0.6135 
Epoch 11/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 828us/step - accuracy: 0.6927 - loss: 0.5942
Epoch 12/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [90]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = loan_model.evaluate(X_test_scaled, y_test, verbose=2)


# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

13/13 - 0s - 17ms/step - accuracy: 0.7450 - loss: 0.5118
Loss: 0.5117834210395813, Accuracy: 0.7450000047683716


### Step 4: Save and export your model to a keras file, and name the file `student_loans.keras`.


In [91]:
# Set the model's file path
path = 'C:/Users/Daniel Quinn/Desktop/Bootcamp/4-Challenge Assignments/neural-network-challenge-1/saved_models/loan_model.keras'

# Export your model to a keras file
loan_model.save(path)

---
## Predict Loan Repayment Success by Using your Neural Network Model

### Step 1: Reload your saved model.

In [92]:
# Set the model's file path
loan_model_path = path

# Load the model to a new object
imported_loan_model = tf.keras.models.load_model(loan_model_path)

### Step 2: Make predictions on the testing data and save the predictions to a DataFrame.

In [98]:
# Make predictions with the test data
prediction = imported_loan_model.predict(X_test_scaled, verbose=2)

# Display a sample of the predictions
prediction

13/13 - 0s - 3ms/step


array([[0.5491177 ],
       [0.2789067 ],
       [0.88774335],
       [0.5879521 ],
       [0.97222245],
       [0.79507923],
       [0.91926426],
       [0.07494063],
       [0.75579727],
       [0.2754807 ],
       [0.8947657 ],
       [0.36807814],
       [0.46319214],
       [0.9376562 ],
       [0.5279119 ],
       [0.3044893 ],
       [0.8147811 ],
       [0.40518633],
       [0.62629914],
       [0.4445746 ],
       [0.6234865 ],
       [0.80693233],
       [0.36366826],
       [0.96007806],
       [0.27760118],
       [0.866129  ],
       [0.8557205 ],
       [0.4774826 ],
       [0.18708983],
       [0.42792928],
       [0.2975688 ],
       [0.9480085 ],
       [0.21882276],
       [0.9370495 ],
       [0.1207206 ],
       [0.6188003 ],
       [0.34924665],
       [0.41233948],
       [0.9179782 ],
       [0.29543293],
       [0.87995964],
       [0.1037389 ],
       [0.15982354],
       [0.8544034 ],
       [0.12456359],
       [0.692658  ],
       [0.21110167],
       [0.329

In [94]:
# Save the predictions to a DataFrame and round the predictions to binary results
prediction_df = pd.DataFrame(columns=["Predictions"], data=prediction)
prediction_df['Predictions'] = round(prediction_df["Predictions"],0)
prediction_df

,Predictions
0,1.0
1,0.0
2,1.0
3,1.0
4,1.0
...,...
395,1.0
396,0.0
397,1.0
398,0.0


### Step 4: Display a classification report with the y test data and predictions

In [95]:
# Print the classification report with the y test data and predictions
print(classification_report(y_test, prediction_df['Predictions'].values))

              precision    recall  f1-score   support

           0       0.72      0.76      0.74       188
           1       0.77      0.74      0.75       212

    accuracy                           0.74       400
   macro avg       0.74      0.75      0.74       400
weighted avg       0.75      0.74      0.75       400



---
## Discuss creating a recommendation system for student loans

Briefly answer the following questions in the space provided:

1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.

2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.

3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.

**1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.**  

 - Residency status - whether or not someone was a US citizen might affect the number and types of loans available (US govt or specific state-sponsored loans might be available to US citizens or residents of specific states only)  
 - Veteran status - many loans exist for veterans of the US military  
 - Racial background - loans specifically for distinct racial groups   
 - Gender - loans specifically for women in specific areas of study (ie: STEM)  
 - Income - many loan programs are needs-based  
 - Family employment history - loans specifically for children of first responders, eg.  
 - Intended field of study - loans and grants exist for students choosing to study certain disciplines  
   
     
     
 **2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.**  
   

 Of the three, I would select the content-based filter for my recommendation engine.  

The applicant would be providing a set of parameters that could be compared against the parameters for each type of loan.  

For example, federal loans, such as Pell Grants (for students with "exceptional financial need") and FAFSA are needs-based, and thus family income would be an appropriate datapoint to have in determining if a student is eligible, and thus a recommendation should be made. Similarly, there are many scholarships or loans to promote women in STEM disciplines, so knowing an applicant's gender would be crucial information.  
  
    
    
 
**3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.**
1. Ensuring that the model doesn't return false positives and false negatives. Improperly telling an applicant that they are/are not eligible for a certain loan or scholarship program casts doubt on the reliability and ultimate utility of such a model.

2. Ensuring not only that the list of loans is current, but that the qualifications are routinekly updated as well. If the eligibility requirements for as loan or grant program becomes more or less stringent, that affects who should apply.  
   Similarly, if a new loan or grant program is launched, and the model isn't updated, users of the model might be missing out on a valuable opportunity, which, again, may cast doubt on the reliability of the model.
